---
date: 2023-08-28
author: Cristian Israel Donato Flores
---

# Algoritmo genetico aplicado a las funciones para maximizar y minimizar funciones

In [130]:
import pandas as pd
import random
import numpy as np
import  math

## Parametros

In [131]:
NUM_CROMOSOMAS = 100
NUM_GENERACIONES = 200
PROB_MUTACION = 0.5
PORCENTAJE_ELITISMO = 0.1

X_MAX = 20
X_MIN = 0
PRECISION = 0.0001
L = round( math.log2(1 + (X_MAX - X_MIN) / PRECISION) )

MAXIMO = 0.5


## Métodos

In [132]:
def crear_cromosoma():
    return [random.randint(0, 1) for _ in range(L)]

def evaluar_fitness(cromosoma:list):
    return sum(cromosoma)

def seleccion(cromosomas):
    return random.choice(cromosomas)

def cruzar(padre1:list, padre2:list):
    punto_cruza = random.randint(1, L - 1)
    hijo1 = padre1[:punto_cruza] + padre2[punto_cruza:]
    hijo2 = padre2[:punto_cruza] + padre1[punto_cruza:]
    return hijo1, hijo2

def mutar(cromosoma:list):
    for i in range(L):
        if(random.random() > PROB_MUTACION):
            cromosoma[i] = 1 - cromosoma[i] # Cambiamos el valor de 1 a 0 y viceversa

def x(c:str):
    return X_MIN + int(c, 2) * ( (X_MAX - X_MIN) / ( (2 ** L) - 1 ) )

def f(x:float):
    return x / (1 + x**2)

def fit(score:float):
    return abs(MAXIMO - score)

## ONE_MAX

In [133]:
poblacion = [crear_cromosoma() for _ in range(NUM_CROMOSOMAS)]

for generacion in range(NUM_GENERACIONES):
    nueva_poblacion = []

    num_elitismo = int(NUM_CROMOSOMAS * PORCENTAJE_ELITISMO)
    nueva_poblacion.extend(sorted(poblacion, key=evaluar_fitness, reverse=True)[:num_elitismo])

    for _ in range(NUM_CROMOSOMAS // 2):
        padre1 = seleccion(poblacion)
        padre2 = seleccion(poblacion)

        hijo1, hijo2 = cruzar(padre1, padre2)

        mutar(hijo1)
        mutar(hijo2)

        nueva_poblacion.extend([hijo1, hijo2])

    poblacion = nueva_poblacion

mejor_cromosoma = max(poblacion, key=evaluar_fitness)
print("Cromosoma óptimo: ", mejor_cromosoma)
print("Fitness: ", evaluar_fitness(mejor_cromosoma))

Cromosoma óptimo:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
Fitness:  17


## Valores de x y adaptación

### DataFrame

In [134]:
poblacion = [''.join(map(str,i)) for i in poblacion]
data = pd.DataFrame({"cromosoma" : poblacion})
data["x"] = data["cromosoma"].apply(x)
data["adaptacion"] = data["x"].apply(f)
data["fitness"] = data["adaptacion"].apply(fit)
data

,cromosoma,x,adaptacion,fitness
0,111111111011111111,19.980469,0.049924,0.450076
1,111111111101111011,19.989929,0.049900,0.450100
2,011111111111111011,9.999657,0.099013,0.400987
3,111111111101111101,19.990082,0.049900,0.450100
4,111111111111110110,19.999313,0.049877,0.450123
...,...,...,...,...
105,010011111000000110,6.211419,0.156926,0.343074
106,101010101101000000,13.344777,0.074517,0.425483
107,110110010111010111,16.989124,0.058658,0.441342
108,101100010100011111,13.850074,0.071827,0.428173


# Resultados

In [135]:
i_min = data["fitness"].idxmin()
print("El mejor cromosoma es:")
print(data.loc[i_min])

El mejor cromosoma es:
cromosoma     000011001111011101
x                       1.012959
adaptacion              0.499959
fitness                 0.000041
Name: 68, dtype: object
